## Markerless Mice Tracking for Social Experiments
This is an implementation of the pipeline described in the paper. 
* Input of the pipeline can be a video or a directory containing frames in sequence. 
* Output of the workflow are two csv files storing coordinates of snout and tailbase of corresponding to two mice: *features_mouse1_ensemble.csv* and *features_mouse2_ensemble.csv*
    

This pipeline used a pretrained Mask-RCNN model and a pretrained DeepLabCut model provided in subfolders mrcnn_models and dlc_models. However, Mask-RCNN and DeepLabCut models should be retrained on new data if the settings of new videos are different with our setting in the paper. The code to train those models can be found in two Jupyter Notebooks as follows:
* *deeplabcut_training.ipynb*
* *mrcnn_training.ipynb*


In [10]:
import os
import sys

# Root directory of the project
ROOT_DIR = os.path.abspath("../")
sys.path.append(ROOT_DIR)  # To find local version of the library

from mouse.utils import labelmejson_to_png
from mouse.utils import video2frames, background_subtraction
from mouse.utils import correct_segmentation_errors, mouse_mrcnn_segmentation, check_mrcnn_model_path
from mouse.utils import tracking_inference, mask_based_detection, mice_separation
from mouse.utils import deeplabcut_detection, ensemble_features


### Path to cage background which is normally imaged before the experiment

In [11]:
background_dir='..\\videos\\BG1.jpg'                                       #----------------update your cage background

### Path the a new video or frames
The input of the pipeline can be a video or a directory containing a sequence of frames. 

In [12]:
# Paths to a new video 
tracking_video_dir = "..\\videos\\video1.avi"                            #----------------update if input is a video

# Extracting the video into frames 
print('The video will be extracted into frames stored in {}'.format(os.path.splitext(tracking_video_dir)[0] + '\\images'))
video2frames(tracking_video_dir)  

frames_dir = os.path.join(os.path.splitext(tracking_video_dir)[0], 'images')

The video will be extracted into frames stored in ..\videos\video1\images


In [ ]:
# Path to frames
#frames_dir = '..\\videos\\video1\images'                                 #-------------------update if input are frames

### Step 1: Conventional foreground detection to segment mice

In [13]:
components = background_subtraction(frames_dir, background_dir)

### Step 2: Mask-RCNN to segment mice in the failed frames in step 1
Update MODEL_PATH for a specific model you want to use. If MODEL_PATH == None, the weights of your last trained model will be loaded

In [15]:
# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "mrcnn_models")

# Directory to load weights of a model. If MRCNN_MODEL_PATH=None, the latest weights in MODEL_DIR will be loaded
MRCNN_MODEL_PATH = "..\\mrcnn_models\\mask_rcnn_mouse_0025.h5"  # None

if MRCNN_MODEL_PATH != None:
    if not os.path.exists(MRCNN_MODEL_PATH):
        print("Please read mrcnn_models/README.md to download our trained model")
    
    else:
        components = mouse_mrcnn_segmentation(components, frames_dir, background_dir, model_dir=MODEL_DIR, model_path=MRCNN_MODEL_PATH)
        
elif MRCNN_MODEL_PATH == None:
    if check_mrcnn_model_path(MODEL_DIR):
        print("The latest trained model will be loaded")
        components = mouse_mrcnn_segmentation(components, frames_dir, background_dir, model_dir=MODEL_DIR, model_path=MRCNN_MODEL_PATH)
    else:
        print("Could not find model directory under {}".format(MODEL_DIR))
        print("Please follow the pipeline in mrcnn_training.ipynb to train your own model, then run this step again.")

The video has 500 frames: 
Processing 1 images
image                    shape: (540, 540, 3)         min:   50.00000  max:  254.00000  uint8
molded_images            shape: (1, 512, 512, 3)      min:  -77.00000  max:  124.00000  float64
image_metas              shape: (1, 14)               min:    0.00000  max:  540.00000  float64
anchors                  shape: (1, 65472, 4)         min:   -0.70849  max:    1.58325  float32
Processing 1 images
image                    shape: (540, 540, 3)         min:   47.00000  max:  255.00000  uint8
molded_images            shape: (1, 512, 512, 3)      min:  -78.00000  max:  125.00000  float64
image_metas              shape: (1, 14)               min:    0.00000  max:  540.00000  float64
anchors                  shape: (1, 65472, 4)         min:   -0.70849  max:    1.58325  float32
Processing 1 images
image                    shape: (540, 540, 3)         min:   44.00000  max:  255.00000  uint8
molded_images            shape: (1, 512, 512, 3)      m

Processing 1 images
image                    shape: (540, 540, 3)         min:   52.00000  max:  255.00000  uint8
molded_images            shape: (1, 512, 512, 3)      min:  -76.00000  max:  126.00000  float64
image_metas              shape: (1, 14)               min:    0.00000  max:  540.00000  float64
anchors                  shape: (1, 65472, 4)         min:   -0.70849  max:    1.58325  float32
Processing 1 images
image                    shape: (540, 540, 3)         min:   50.00000  max:  255.00000  uint8
molded_images            shape: (1, 512, 512, 3)      min:  -76.00000  max:  126.00000  float64
image_metas              shape: (1, 14)               min:    0.00000  max:  540.00000  float64
anchors                  shape: (1, 65472, 4)         min:   -0.70849  max:    1.58325  float32
Processing 1 images
image                    shape: (540, 540, 3)         min:   54.00000  max:  255.00000  uint8
molded_images            shape: (1, 512, 512, 3)      min:  -74.00000  max:  126.0

Processing 1 images
image                    shape: (540, 540, 3)         min:   43.00000  max:  255.00000  uint8
molded_images            shape: (1, 512, 512, 3)      min:  -85.00000  max:  126.00000  float64
image_metas              shape: (1, 14)               min:    0.00000  max:  540.00000  float64
anchors                  shape: (1, 65472, 4)         min:   -0.70849  max:    1.58325  float32
Processing 1 images
image                    shape: (540, 540, 3)         min:   45.00000  max:  255.00000  uint8
molded_images            shape: (1, 512, 512, 3)      min:  -83.00000  max:  125.00000  float64
image_metas              shape: (1, 14)               min:    0.00000  max:  540.00000  float64
anchors                  shape: (1, 65472, 4)         min:   -0.70849  max:    1.58325  float32
Processing 1 images
image                    shape: (540, 540, 3)         min:   44.00000  max:  255.00000  uint8
molded_images            shape: (1, 512, 512, 3)      min:  -84.00000  max:  126.0

Processing 1 images
image                    shape: (540, 540, 3)         min:   51.00000  max:  255.00000  uint8
molded_images            shape: (1, 512, 512, 3)      min:  -77.00000  max:  125.00000  float64
image_metas              shape: (1, 14)               min:    0.00000  max:  540.00000  float64
anchors                  shape: (1, 65472, 4)         min:   -0.70849  max:    1.58325  float32
Processing 1 images
image                    shape: (540, 540, 3)         min:   50.00000  max:  255.00000  uint8
molded_images            shape: (1, 512, 512, 3)      min:  -78.00000  max:  126.00000  float64
image_metas              shape: (1, 14)               min:    0.00000  max:  540.00000  float64
anchors                  shape: (1, 65472, 4)         min:   -0.70849  max:    1.58325  float32
Processing 1 images
image                    shape: (540, 540, 3)         min:   48.00000  max:  255.00000  uint8
molded_images            shape: (1, 512, 512, 3)      min:  -78.00000  max:  125.0

Processing 1 images
image                    shape: (540, 540, 3)         min:   47.00000  max:  255.00000  uint8
molded_images            shape: (1, 512, 512, 3)      min:  -80.00000  max:  125.00000  float64
image_metas              shape: (1, 14)               min:    0.00000  max:  540.00000  float64
anchors                  shape: (1, 65472, 4)         min:   -0.70849  max:    1.58325  float32
Processing 1 images
image                    shape: (540, 540, 3)         min:   47.00000  max:  255.00000  uint8
molded_images            shape: (1, 512, 512, 3)      min:  -80.00000  max:  126.00000  float64
image_metas              shape: (1, 14)               min:    0.00000  max:  540.00000  float64
anchors                  shape: (1, 65472, 4)         min:   -0.70849  max:    1.58325  float32
Processing 1 images
image                    shape: (540, 540, 3)         min:   48.00000  max:  254.00000  uint8
molded_images            shape: (1, 512, 512, 3)      min:  -79.00000  max:  125.0

...

Processing 1 images
image                    shape: (540, 540, 3)         min:   48.00000  max:  255.00000  uint8
molded_images            shape: (1, 512, 512, 3)      min:  -80.00000  max:  126.00000  float64
image_metas              shape: (1, 14)               min:    0.00000  max:  540.00000  float64
anchors                  shape: (1, 65472, 4)         min:   -0.70849  max:    1.58325  float32
Processing 1 images
image                    shape: (540, 540, 3)         min:   48.00000  max:  255.00000  uint8
molded_images            shape: (1, 512, 512, 3)      min:  -77.00000  max:  125.00000  float64
image_metas              shape: (1, 14)               min:    0.00000  max:  540.00000  float64
anchors                  shape: (1, 65472, 4)         min:   -0.70849  max:    1.58325  float32
Processing 1 images
image                    shape: (540, 540, 3)         min:   47.00000  max:  255.00000  uint8
molded_images            shape: (1, 512, 512, 3)      min:  -79.00000  max:  126.0

Processing 1 images
image                    shape: (540, 540, 3)         min:   45.00000  max:  255.00000  uint8
molded_images            shape: (1, 512, 512, 3)      min:  -83.00000  max:  126.00000  float64
image_metas              shape: (1, 14)               min:    0.00000  max:  540.00000  float64
anchors                  shape: (1, 65472, 4)         min:   -0.70849  max:    1.58325  float32
Processing 1 images
image                    shape: (540, 540, 3)         min:   47.00000  max:  255.00000  uint8
molded_images            shape: (1, 512, 512, 3)      min:  -81.00000  max:  125.00000  float64
image_metas              shape: (1, 14)               min:    0.00000  max:  540.00000  float64
anchors                  shape: (1, 65472, 4)         min:   -0.70849  max:    1.58325  float32
Processing 1 images
image                    shape: (540, 540, 3)         min:   45.00000  max:  255.00000  uint8
molded_images            shape: (1, 512, 512, 3)      min:  -83.00000  max:  126.0

Processing 1 images
image                    shape: (540, 540, 3)         min:   48.00000  max:  255.00000  uint8
molded_images            shape: (1, 512, 512, 3)      min:  -79.00000  max:  126.00000  float64
image_metas              shape: (1, 14)               min:    0.00000  max:  540.00000  float64
anchors                  shape: (1, 65472, 4)         min:   -0.70849  max:    1.58325  float32
Processing 1 images
image                    shape: (540, 540, 3)         min:   49.00000  max:  254.00000  uint8
molded_images            shape: (1, 512, 512, 3)      min:  -77.00000  max:  124.00000  float64
image_metas              shape: (1, 14)               min:    0.00000  max:  540.00000  float64
anchors                  shape: (1, 65472, 4)         min:   -0.70849  max:    1.58325  float32
Processing 1 images
image                    shape: (540, 540, 3)         min:   49.00000  max:  255.00000  uint8
molded_images            shape: (1, 512, 512, 3)      min:  -78.00000  max:  126.0

Processing 1 images
image                    shape: (540, 540, 3)         min:   49.00000  max:  254.00000  uint8
molded_images            shape: (1, 512, 512, 3)      min:  -79.00000  max:  124.00000  float64
image_metas              shape: (1, 14)               min:    0.00000  max:  540.00000  float64
anchors                  shape: (1, 65472, 4)         min:   -0.70849  max:    1.58325  float32
Processing 1 images
image                    shape: (540, 540, 3)         min:   49.00000  max:  255.00000  uint8
molded_images            shape: (1, 512, 512, 3)      min:  -79.00000  max:  125.00000  float64
image_metas              shape: (1, 14)               min:    0.00000  max:  540.00000  float64
anchors                  shape: (1, 65472, 4)         min:   -0.70849  max:    1.58325  float32
Processing 1 images
image                    shape: (540, 540, 3)         min:   48.00000  max:  255.00000  uint8
molded_images            shape: (1, 512, 512, 3)      min:  -80.00000  max:  125.0

Processing 1 images
image                    shape: (540, 540, 3)         min:   48.00000  max:  255.00000  uint8
molded_images            shape: (1, 512, 512, 3)      min:  -80.00000  max:  126.00000  float64
image_metas              shape: (1, 14)               min:    0.00000  max:  540.00000  float64
anchors                  shape: (1, 65472, 4)         min:   -0.70849  max:    1.58325  float32
Processing 1 images
image                    shape: (540, 540, 3)         min:   47.00000  max:  255.00000  uint8
molded_images            shape: (1, 512, 512, 3)      min:  -79.00000  max:  126.00000  float64
image_metas              shape: (1, 14)               min:    0.00000  max:  540.00000  float64
anchors                  shape: (1, 65472, 4)         min:   -0.70849  max:    1.58325  float32


### Step 3: Suggesting intervention
Althought we did not intervene the Mask-RCNN results in the paper, it is recommended to fix very 3 consecutive failed frames. 
Skip step 3 if you don't want to correct the mistakes.

To correct segmentation errors, in Labelme GUI open *fix_dir* and annotate both mice as a label of *mouse*

In [16]:
fix_dir = os.path.join(os.path.dirname(frames_dir), 'suggested_fix')
os.mkdir(fix_dir)

suggesting_correction = correct_segmentation_errors(components, fix_dir, frames_dir)

if suggesting_correction==0:
    print('There are no frames for correcting')
else:
    print('{} frames in the directory {} are recommended to be corrected using Labelme'.format(suggesting_correction, fix_dir))
    ! labelme

There are no frames for correcting


In [17]:
# convert fix_frames back to FG and update components
output_dir = os.path.join(os.path.dirname(frames_dir),'FG')        
labelmejson_to_png(fix_dir, output_dir)      

### Step 4: Tracking inference

In [18]:
fg_dir = os.path.join(os.path.dirname(frames_dir),'FG')
tracking_inference(fg_dir, components)

C:\Users\vananh\AppData\Local\conda\conda\envs\markerless-mice-tracking-windowsGPU\lib\site-packages\skimage\util\dtype.py:141: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


### Step 5:  Mask-based detection

In [19]:
tracking_dir = os.path.join(os.path.dirname(frames_dir),'tracking')
features_mouse1_md, features_mouse2_md = mask_based_detection(tracking_dir, components)

### Step 6: Deeplabcut detection
Update config_dir to load your own trained model.

In [20]:
# Separating the video into 2 videos corresponding to two animals.
mice_separation(tracking_dir, frames_dir, background_dir)   

C:\Users\vananh\AppData\Local\conda\conda\envs\markerless-mice-tracking-windowsGPU\lib\site-packages\skimage\util\dtype.py:137: UserWarning: Possible sign loss when converting negative image of type int32 to positive image of type uint8.
  .format(dtypeobj_in, dtypeobj_out))
C:\Users\vananh\AppData\Local\conda\conda\envs\markerless-mice-tracking-windowsGPU\lib\site-packages\skimage\util\dtype.py:190: UserWarning: Downcasting int32 to uint8 without scaling because max value 255 fits in uint8
  "value {} fits in {}".format(a.dtype, dtype, a.max(), dtype))
C:\Users\vananh\AppData\Local\conda\conda\envs\markerless-mice-tracking-windowsGPU\lib\site-packages\skimage\util\dtype.py:137: UserWarning: Possible sign loss when converting negative image of type int32 to positive image of type uint8.
  .format(dtypeobj_in, dtypeobj_out))
C:\Users\vananh\AppData\Local\conda\conda\envs\markerless-mice-tracking-windowsGPU\lib\site-packages\skimage\util\dtype.py:190: UserWarning: Downcasting int32 to ui

C:\Users\vananh\AppData\Local\conda\conda\envs\markerless-mice-tracking-windowsGPU\lib\site-packages\skimage\util\dtype.py:137: UserWarning: Possible sign loss when converting negative image of type int32 to positive image of type uint8.
  .format(dtypeobj_in, dtypeobj_out))
C:\Users\vananh\AppData\Local\conda\conda\envs\markerless-mice-tracking-windowsGPU\lib\site-packages\skimage\util\dtype.py:190: UserWarning: Downcasting int32 to uint8 without scaling because max value 255 fits in uint8
  "value {} fits in {}".format(a.dtype, dtype, a.max(), dtype))
C:\Users\vananh\AppData\Local\conda\conda\envs\markerless-mice-tracking-windowsGPU\lib\site-packages\skimage\util\dtype.py:137: UserWarning: Possible sign loss when converting negative image of type int32 to positive image of type uint8.
  .format(dtypeobj_in, dtypeobj_out))
C:\Users\vananh\AppData\Local\conda\conda\envs\markerless-mice-tracking-windowsGPU\lib\site-packages\skimage\util\dtype.py:190: UserWarning: Downcasting int32 to ui

C:\Users\vananh\AppData\Local\conda\conda\envs\markerless-mice-tracking-windowsGPU\lib\site-packages\skimage\util\dtype.py:137: UserWarning: Possible sign loss when converting negative image of type int32 to positive image of type uint8.
  .format(dtypeobj_in, dtypeobj_out))
C:\Users\vananh\AppData\Local\conda\conda\envs\markerless-mice-tracking-windowsGPU\lib\site-packages\skimage\util\dtype.py:190: UserWarning: Downcasting int32 to uint8 without scaling because max value 255 fits in uint8
  "value {} fits in {}".format(a.dtype, dtype, a.max(), dtype))
C:\Users\vananh\AppData\Local\conda\conda\envs\markerless-mice-tracking-windowsGPU\lib\site-packages\skimage\util\dtype.py:137: UserWarning: Possible sign loss when converting negative image of type int32 to positive image of type uint8.
  .format(dtypeobj_in, dtypeobj_out))
C:\Users\vananh\AppData\Local\conda\conda\envs\markerless-mice-tracking-windowsGPU\lib\site-packages\skimage\util\dtype.py:190: UserWarning: Downcasting int32 to ui

C:\Users\vananh\AppData\Local\conda\conda\envs\markerless-mice-tracking-windowsGPU\lib\site-packages\skimage\util\dtype.py:137: UserWarning: Possible sign loss when converting negative image of type int32 to positive image of type uint8.
  .format(dtypeobj_in, dtypeobj_out))
C:\Users\vananh\AppData\Local\conda\conda\envs\markerless-mice-tracking-windowsGPU\lib\site-packages\skimage\util\dtype.py:190: UserWarning: Downcasting int32 to uint8 without scaling because max value 255 fits in uint8
  "value {} fits in {}".format(a.dtype, dtype, a.max(), dtype))
C:\Users\vananh\AppData\Local\conda\conda\envs\markerless-mice-tracking-windowsGPU\lib\site-packages\skimage\util\dtype.py:137: UserWarning: Possible sign loss when converting negative image of type int32 to positive image of type uint8.
  .format(dtypeobj_in, dtypeobj_out))
C:\Users\vananh\AppData\Local\conda\conda\envs\markerless-mice-tracking-windowsGPU\lib\site-packages\skimage\util\dtype.py:190: UserWarning: Downcasting int32 to ui

C:\Users\vananh\AppData\Local\conda\conda\envs\markerless-mice-tracking-windowsGPU\lib\site-packages\skimage\util\dtype.py:137: UserWarning: Possible sign loss when converting negative image of type int32 to positive image of type uint8.
  .format(dtypeobj_in, dtypeobj_out))
C:\Users\vananh\AppData\Local\conda\conda\envs\markerless-mice-tracking-windowsGPU\lib\site-packages\skimage\util\dtype.py:190: UserWarning: Downcasting int32 to uint8 without scaling because max value 255 fits in uint8
  "value {} fits in {}".format(a.dtype, dtype, a.max(), dtype))
C:\Users\vananh\AppData\Local\conda\conda\envs\markerless-mice-tracking-windowsGPU\lib\site-packages\skimage\util\dtype.py:137: UserWarning: Possible sign loss when converting negative image of type int32 to positive image of type uint8.
  .format(dtypeobj_in, dtypeobj_out))
C:\Users\vananh\AppData\Local\conda\conda\envs\markerless-mice-tracking-windowsGPU\lib\site-packages\skimage\util\dtype.py:190: UserWarning: Downcasting int32 to ui

...

C:\Users\vananh\AppData\Local\conda\conda\envs\markerless-mice-tracking-windowsGPU\lib\site-packages\skimage\util\dtype.py:137: UserWarning: Possible sign loss when converting negative image of type int32 to positive image of type uint8.
  .format(dtypeobj_in, dtypeobj_out))
C:\Users\vananh\AppData\Local\conda\conda\envs\markerless-mice-tracking-windowsGPU\lib\site-packages\skimage\util\dtype.py:190: UserWarning: Downcasting int32 to uint8 without scaling because max value 255 fits in uint8
  "value {} fits in {}".format(a.dtype, dtype, a.max(), dtype))
C:\Users\vananh\AppData\Local\conda\conda\envs\markerless-mice-tracking-windowsGPU\lib\site-packages\skimage\util\dtype.py:137: UserWarning: Possible sign loss when converting negative image of type int32 to positive image of type uint8.
  .format(dtypeobj_in, dtypeobj_out))
C:\Users\vananh\AppData\Local\conda\conda\envs\markerless-mice-tracking-windowsGPU\lib\site-packages\skimage\util\dtype.py:190: UserWarning: Downcasting int32 to ui

C:\Users\vananh\AppData\Local\conda\conda\envs\markerless-mice-tracking-windowsGPU\lib\site-packages\skimage\util\dtype.py:137: UserWarning: Possible sign loss when converting negative image of type int32 to positive image of type uint8.
  .format(dtypeobj_in, dtypeobj_out))
C:\Users\vananh\AppData\Local\conda\conda\envs\markerless-mice-tracking-windowsGPU\lib\site-packages\skimage\util\dtype.py:190: UserWarning: Downcasting int32 to uint8 without scaling because max value 254 fits in uint8
  "value {} fits in {}".format(a.dtype, dtype, a.max(), dtype))
C:\Users\vananh\AppData\Local\conda\conda\envs\markerless-mice-tracking-windowsGPU\lib\site-packages\skimage\util\dtype.py:137: UserWarning: Possible sign loss when converting negative image of type int32 to positive image of type uint8.
  .format(dtypeobj_in, dtypeobj_out))
C:\Users\vananh\AppData\Local\conda\conda\envs\markerless-mice-tracking-windowsGPU\lib\site-packages\skimage\util\dtype.py:190: UserWarning: Downcasting int32 to ui

C:\Users\vananh\AppData\Local\conda\conda\envs\markerless-mice-tracking-windowsGPU\lib\site-packages\skimage\util\dtype.py:137: UserWarning: Possible sign loss when converting negative image of type int32 to positive image of type uint8.
  .format(dtypeobj_in, dtypeobj_out))
C:\Users\vananh\AppData\Local\conda\conda\envs\markerless-mice-tracking-windowsGPU\lib\site-packages\skimage\util\dtype.py:190: UserWarning: Downcasting int32 to uint8 without scaling because max value 255 fits in uint8
  "value {} fits in {}".format(a.dtype, dtype, a.max(), dtype))
C:\Users\vananh\AppData\Local\conda\conda\envs\markerless-mice-tracking-windowsGPU\lib\site-packages\skimage\util\dtype.py:137: UserWarning: Possible sign loss when converting negative image of type int32 to positive image of type uint8.
  .format(dtypeobj_in, dtypeobj_out))
C:\Users\vananh\AppData\Local\conda\conda\envs\markerless-mice-tracking-windowsGPU\lib\site-packages\skimage\util\dtype.py:190: UserWarning: Downcasting int32 to ui

C:\Users\vananh\AppData\Local\conda\conda\envs\markerless-mice-tracking-windowsGPU\lib\site-packages\skimage\util\dtype.py:137: UserWarning: Possible sign loss when converting negative image of type int32 to positive image of type uint8.
  .format(dtypeobj_in, dtypeobj_out))
C:\Users\vananh\AppData\Local\conda\conda\envs\markerless-mice-tracking-windowsGPU\lib\site-packages\skimage\util\dtype.py:190: UserWarning: Downcasting int32 to uint8 without scaling because max value 255 fits in uint8
  "value {} fits in {}".format(a.dtype, dtype, a.max(), dtype))
C:\Users\vananh\AppData\Local\conda\conda\envs\markerless-mice-tracking-windowsGPU\lib\site-packages\skimage\util\dtype.py:137: UserWarning: Possible sign loss when converting negative image of type int32 to positive image of type uint8.
  .format(dtypeobj_in, dtypeobj_out))
C:\Users\vananh\AppData\Local\conda\conda\envs\markerless-mice-tracking-windowsGPU\lib\site-packages\skimage\util\dtype.py:190: UserWarning: Downcasting int32 to ui

C:\Users\vananh\AppData\Local\conda\conda\envs\markerless-mice-tracking-windowsGPU\lib\site-packages\skimage\util\dtype.py:137: UserWarning: Possible sign loss when converting negative image of type int32 to positive image of type uint8.
  .format(dtypeobj_in, dtypeobj_out))
C:\Users\vananh\AppData\Local\conda\conda\envs\markerless-mice-tracking-windowsGPU\lib\site-packages\skimage\util\dtype.py:190: UserWarning: Downcasting int32 to uint8 without scaling because max value 255 fits in uint8
  "value {} fits in {}".format(a.dtype, dtype, a.max(), dtype))
C:\Users\vananh\AppData\Local\conda\conda\envs\markerless-mice-tracking-windowsGPU\lib\site-packages\skimage\util\dtype.py:137: UserWarning: Possible sign loss when converting negative image of type int32 to positive image of type uint8.
  .format(dtypeobj_in, dtypeobj_out))
C:\Users\vananh\AppData\Local\conda\conda\envs\markerless-mice-tracking-windowsGPU\lib\site-packages\skimage\util\dtype.py:190: UserWarning: Downcasting int32 to ui

In [21]:
# Detect the key points by Deeplabcut model
config_dir = '../dlc_models/dlc_mice_tracking/config.yaml'              #------Update this for your own model

if os.path.exists(config_dir):
    video_dir = [os.path.join(os.path.dirname(tracking_dir), 'mouse1.avi'),
                 os.path.join(os.path.dirname(tracking_dir), 'mouse2.avi')]

    features_mouse1_dlc, features_mouse2_dlc = deeplabcut_detection(config_dir, video_dir)
else:
    print("Please read dlc_models/README.md to download our trained model and run this step again")
    print("Or follow the pipeline in dlc_training.ipynb to train our own model, then update config_dir and run this step again.")

Using snapshot-1030000 for model ..\dlc_models\dlc_mice_tracking\dlc-models\iteration-0\TrackingAug13-trainset80shuffle1
num_outputs =  1
Instructions for updating:
Use standard file APIs to check for files with this prefix.


Instructions for updating:
Use standard file APIs to check for files with this prefix.


INFO:tensorflow:Restoring parameters from ..\dlc_models\dlc_mice_tracking\dlc-models\iteration-0\TrackingAug13-trainset80shuffle1\train\snapshot-1030000


INFO:tensorflow:Restoring parameters from ..\dlc_models\dlc_mice_tracking\dlc-models\iteration-0\TrackingAug13-trainset80shuffle1\train\snapshot-1030000
  0%|                                                  | 0/500 [00:00<?, ?it/s]

Starting to analyze %  ..\videos\video1\mouse1.avi
Loading  ..\videos\video1\mouse1.avi
Duration of video [s]:  16.67 , recorded with  30.0 fps!
Overall # of frames:  500  found with (before cropping) frame dimensions:  540 540
Starting to extract posture


...

Saving results in ..\videos\video1...
Starting to analyze %  ..\videos\video1\mouse2.avi
Loading  ..\videos\video1\mouse2.avi
Duration of video [s]:  16.67 , recorded with  30.0 fps!
Overall # of frames:  500  found with (before cropping) frame dimensions:  540 540
Starting to extract posture


510it [05:05,  1.70it/s]                                                       

Detected frames:  500


510it [05:10,  1.64it/s]

Saving results in ..\videos\video1...
The videos are analyzed. Now your research can truly start! 
 You can create labeled videos with 'create_labeled_video'.
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract any outlier frames!


### Step 7:  Ensemble
The output of the pipeline is the coordinates of snout and tailbase of two mice which are saved as features_mouse1_ensemble.csv and features_mouse2_ensemble.csv

In [22]:
features_mouse1_ensemble = ensemble_features(features_mouse1_md, features_mouse1_dlc, tracking_dir, mouse_id=1)    
features_mouse2_ensemble = ensemble_features(features_mouse2_md, features_mouse2_dlc, tracking_dir, mouse_id=2) 

print("Pls find the output results in the directory {}".format(os.path.dirname(frames_dir)))

Pls find the output results in the directory ..\videos\video1
